In [ ]:
import os, sys

notebook_path = os.path.dirname(os.path.abspath("__file__"))
# the code path is two folders up from this notebook + /code
core_path = os.path.dirname(notebook_path)
basepath = os.path.dirname(os.path.dirname(notebook_path))

sys.path.append(core_path)
sys.path.append(basepath)

from core.klusta_functions import klusta
from core.klusta_utils import session_analyzable, find_tetrodes
from core.Tint_Matlab import get_setfile_parameter
from core.defaultParameters import get_default_settings, calculateUseFeatures
from core.delete_temp import get_temp_files

# Define Settings

In [4]:
settings = get_default_settings()

# the default naming structure is the directory\session_basename_tetrodeNumber.cut
# for example: C:\Example\ExampleSession_1.cut for tetrode 1.
# However, there may be a cause where you want to re-analyze with different cut file nomenclature
# so we will allow you to append a value to the end to produce the following:
# directory\session_basename[appendValue]_tetrodeNumber.cut
# so if we append 'TEST' the output would be: C:\Example\ExampleSessionTEST_1.cut for tetrode 1
# set the cut_append value to None for the default
cut_append='KK'  # custom append value
# cut_append=None  # uncomment this for default value

# channels to use:
settings['1'] = 1  # defaults to 1, which is on
settings['2'] = 1  # defaults to 1, which is on
settings['3'] = 1  # defaults to 1, which is on
settings['4'] = 1  # defaults to 1, which is on

# features to use:
settings['PC1'] = 1  # defaults to 1, which is on
settings['PC2'] = 1  # defaults to 1, which is on
settings['PC3'] = 1  # defaults to 1, which is on
settings['PC4'] = 0  # defaults to 0, which is off
settings['A'] = 0  # defaults to 0, which is off
settings['Vt'] = 0  # defaults to 0, which is off
settings['P'] = 0  # defaults to 0, which is off
settings['T'] = 0  # defaults to 0, which is off
settings['tP'] = 0  # defaults to 0, which is off
settings['tT'] = 0  # defaults to 0, which is off
settings['En'] = 0  # defaults to 0, which is off
settings['Ar'] = 0  # defaults to 0, which is off

# re-caculating the feature masks (if channel 2 is off, we will make sure to ignore
# those features)
UseFeatures, numFeats = calculateUseFeatures(settings)
settings['UseFeatures'] = UseFeatures  # feature mask
settings['NumFeat'] = numFeats  # number of features used

# Advanced KlustaKwik Parameters
settings['MaxPos'] = 30  # default 3
settings['nStarts'] = 1  # default = 1
settings['RandomSeed'] = 1  # default = 1
settings['DistThresh'] = 6.907755  # default is 6.907755
settings['FullStepEvery'] = 20  # default is 20
settings['ChangedThresh'] = 0.05  # default is 0.05
settings['MaxIter'] = 500  # default is 500
settings['SplitEvery'] = 40  # default is 40
settings['Subset'] = 1 # default is 1
settings['PenaltyK'] = 1.0  # default is 1.0
settings['PenaltyKLogN'] = 0.0  # default is 0.0

# Reporting Values
settings['Verbose'] = 1  # default of 1
settings['Screen'] = 1  # default of 1
settings['Log File'] = 1  # default of 1


# Misc Settings
settings['Silent'] = 0   # the KK windows will pop up
# number of simultaneous threads to analyze, defaults to 1, uncomment to override
settings['NumThreads'] = 4  # default is 1
settings['Cores'] = os.cpu_count()

# Experimenter Settings
experimenter_settings = {
    'Geoff' : 'geoffrey.m.barrett@gmail.com' # can do [email1@.., email2@..] if you want it sent to more than 1
}

# Email Settings
smtp_settings = {}
smtp_settings['Notification'] = 0 #  1 for send e-mails, 0 for don't send

# if you have the notifications set to 0 you don't have to worry about this.
# we will need an e-mail to send these experimenter's e-mails from
smtp_settings['Username'] = 'example@gmail.com'
smtp_settings['Password'] = 'password'  # associated password
smtp_settings['ServerName'] = 'smtp.gmail.com' # the smtp server name, 'smtp.gmail.com' for gmail
smtp_settings['Port'] = 587  # 587 default for gmail

# Analyze Directory

In [8]:
batch = False

if batch:
    settings['nonbatch'] = 0
else:
    settings['nonbatch'] = 1

settings['move_processed'] = 0  # set to 1 if you want to move to "processed" folder
    
directory = r'E:\Apollo_D_Drive\data\VirtualMazeData\ANT1_2\ParallelLinearGlobalTrack'
directory = r'E:\Apollo_D_Drive\data\VirtualMazeData\NT_360a_2\ParallelLinearGlobalTrack'
directory = r'E:\Apollo_D_Drive\data\VirtualMazeData\NT_181a_2\ParallelLinearGlobalTrack'
# directory = r'C:\Users\Taub Institute\Desktop\test_batchtint'
# directory = r'E:\Apollo_D_Drive\data\MSData\AxonaVsIntan\b6_august_1\Axona'
# directory = r'E:\Apollo_D_Drive\data\VirtualMazeData\NT_361a_2\ParallelLinearGlobalTrack'
# directory = r'C:\Users\Taub Institute\Desktop\test_batchtint'

# Finds Sessions

In [9]:
if batch:
    sub_directories = [d for d in os.listdir(directory)
                                   if os.path.isdir(os.path.join(directory, d)) and
                                   d not in ['Processed', 'Converted']]  # finds the subdirectories within each folder

else:
    sub_directories = [os.path.basename(directory)]
    directory = os.path.dirname(directory)

set_files = []
if len(sub_directories) > 0:
    [[set_files.append(os.path.join(directory, d, file)) for file in os.listdir(os.path.join(directory, d))
                             if '.set' in file and
                             not os.path.isdir(os.path.join(directory, d, file))] for d in sub_directories]

In [10]:
print('There are %d total set files found!' % (len(set_files)))

There are 22 total set files found!


In [11]:
single_file = False
if single_file:
    basename = 'NT_360a_2_1000_plgt_190403_103311_ms'
    set_files = [file for file in set_files if basename in file]

In [12]:
# checking that the set files are analyzable (have the appropriate files, or don't have
# existing cut values already. Essentially all it needs are the tetrode files.
if set_files:
    set_files = [file for file in set_files if session_analyzable(os.path.dirname(file), 
                                                             os.path.splitext(
                                                                 os.path.basename(file))[0],
                                                            append=cut_append)]

In [13]:
print('There are %d un-analyzed set files found!' % (len(set_files)))

There are 22 un-analyzed set files found!


In [14]:
analyzed_set_files = klusta(set_files, settings, 
       smtp_settings=smtp_settings, 
       experimenter_settings=experimenter_settings,
                           append=cut_append)

[2019-04-22 10:49:55]: There are 22 '.set' files to analyze!
[2019-04-22 10:49:55]: Now analyzing tetrodes associated with the NT_181a_2_1000_plgt_190401_151259_ms '.set' file (1/22)!
[2019-04-22 10:49:55]: Now analyzing the following file: NT_181a_2_1000_plgt_190401_151259_ms.1!
[2019-04-22 10:49:55]: Now analyzing the following file: NT_181a_2_1000_plgt_190401_151259_ms.2!
[2019-04-22 10:49:55]: Now analyzing the following file: NT_181a_2_1000_plgt_190401_151259_ms.3!
[2019-04-22 10:49:55]: Now analyzing the following file: NT_181a_2_1000_plgt_190401_151259_ms.4!
[2019-04-22 10:50:33]: The analysis of the NT_181a_2_1000_plgt_190401_151259_ms.1 file has finished!
[2019-04-22 10:50:48]: The analysis of the NT_181a_2_1000_plgt_190401_151259_ms.4 file has finished!
[2019-04-22 10:52:37]: The analysis of the NT_181a_2_1000_plgt_190401_151259_ms.2 file has finished!
[2019-04-22 10:52:56]: The analysis of the NT_181a_2_1000_plgt_190401_151259_ms.3 file has finished!
[2019-04-22 10:52:56]: N

[2019-04-22 11:34:05]: The analysis of the NT_181a_2_1000_plgt_190408_144305_ms.3 file has finished!
[2019-04-22 11:34:05]: Now analyzing tetrodes associated with the NT_181a_2_1000_plgt_190409_111241_ms '.set' file (10/22)!
[2019-04-22 11:34:05]: Now analyzing the following file: NT_181a_2_1000_plgt_190409_111241_ms.1!
[2019-04-22 11:34:05]: Now analyzing the following file: NT_181a_2_1000_plgt_190409_111241_ms.2!
[2019-04-22 11:34:05]: Now analyzing the following file: NT_181a_2_1000_plgt_190409_111241_ms.3!
[2019-04-22 11:34:05]: Now analyzing the following file: NT_181a_2_1000_plgt_190409_111241_ms.4!
[2019-04-22 11:35:56]: The analysis of the NT_181a_2_1000_plgt_190409_111241_ms.1 file has finished!
[2019-04-22 11:37:48]: The analysis of the NT_181a_2_1000_plgt_190409_111241_ms.2 file has finished!
[2019-04-22 11:40:58]: The analysis of the NT_181a_2_1000_plgt_190409_111241_ms.3 file has finished!
[2019-04-22 11:42:33]: The analysis of the NT_181a_2_1000_plgt_190409_111241_ms.4 fi

[2019-04-22 12:42:10]: The analysis of the NT_181a_2_1025_plgt_190416_134746_ms.4 file has finished!
[2019-04-22 12:42:10]: Now analyzing tetrodes associated with the NT_181a_2_1025_PLGT_190417_095034_ms '.set' file (19/22)!
[2019-04-22 12:42:10]: Now analyzing the following file: NT_181a_2_1025_PLGT_190417_095034_ms.1!
[2019-04-22 12:42:10]: Now analyzing the following file: NT_181a_2_1025_PLGT_190417_095034_ms.2!
[2019-04-22 12:42:10]: Now analyzing the following file: NT_181a_2_1025_PLGT_190417_095034_ms.3!
[2019-04-22 12:42:10]: Now analyzing the following file: NT_181a_2_1025_PLGT_190417_095034_ms.4!
[2019-04-22 12:45:05]: The analysis of the NT_181a_2_1025_PLGT_190417_095034_ms.1 file has finished!
[2019-04-22 12:45:27]: The analysis of the NT_181a_2_1025_PLGT_190417_095034_ms.2 file has finished!
[2019-04-22 12:48:18]: The analysis of the NT_181a_2_1025_PLGT_190417_095034_ms.3 file has finished!
[2019-04-22 12:48:25]: The analysis of the NT_181a_2_1025_PLGT_190417_095034_ms.4 fi

# Delete Temp Files
There are files that KlustaKwik creates that is not necessary after the klusta has been performed, so we will delete them if you set delete_temp to True

In [15]:
delete_temp = True

if delete_temp:
    
    for file in set_files:
        set_path = os.path.splitext(file)[0]
        temp_files = get_temp_files(set_path, append=cut_append)
        print('%d files to delete for the following set path: %s' % (len(temp_files), set_path))
        for _f in temp_files:
            os.remove(_f)

20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_181a_2\ParallelLinearGlobalTrack\NT_181a_2_1000_plgt_190401_151259_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_181a_2\ParallelLinearGlobalTrack\NT_181a_2_1000_plgt_190402_091225_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_181a_2\ParallelLinearGlobalTrack\NT_181a_2_1000_plgt_190402_143546_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_181a_2\ParallelLinearGlobalTrack\NT_181a_2_1000_plgt_190403_123057_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_181a_2\ParallelLinearGlobalTrack\NT_181a_2_1000_plgt_190404_111335_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_181a_2\ParallelLinearGlobalTrack\NT_181a_2_1000_plgt_190404_142810_ms
20 files to delete for the following set path:

# delete all temp files

In [7]:
all_set_files = []
if len(sub_directories) > 0:
    [[all_set_files.append(os.path.join(directory, d, file)) for file in os.listdir(os.path.join(directory, d))
                             if '.set' in file and
                             not os.path.isdir(os.path.join(directory, d, file))] for d in sub_directories]

In [8]:
delete_all_temp = True

if delete_all_temp:
    
    for file in all_set_files:
        set_path = os.path.splitext(file)[0]
        temp_files = get_temp_files(set_path, append=cut_append)
        print('%d files to delete for the following set path: %s' % (len(temp_files), set_path))
        for _f in temp_files:
            os.remove(_f)

20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_360a_2\ParallelLinearGlobalTrack\NT_360a_1_1025_plgt_190415_121659_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_360a_2\ParallelLinearGlobalTrack\NT_360a_1_1025_plgt_190415_144038_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_360a_2\ParallelLinearGlobalTrack\NT_360a_2_1000_plgt_190401_154208_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_360a_2\ParallelLinearGlobalTrack\NT_360a_2_1000_plgt_190402_094238_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_360a_2\ParallelLinearGlobalTrack\NT_360a_2_1000_plgt_190402_133222_ms
20 files to delete for the following set path: E:\Apollo_D_Drive\data\VirtualMazeData\NT_360a_2\ParallelLinearGlobalTrack\NT_360a_2_1000_plgt_190403_103311_ms
19 files to delete for the following set path: